In [1]:
from ast import literal_eval

import numpy as np
import pandas as pd
import scipy.sparse

from lightfm import LightFM
from lightfm.data import Dataset
from recommender.testing.custom_metric_utils import create_recommendations_profiles_embeddings, calculate_metric_scores
from recommender.testing.custom_metric_utils import create_users_profiles_embeddings
from recommender.testing.dataset_utils import prepare_interactions
from recommender.tools.lightfm_utils import LightFMTests

In [2]:
DATA_ROOT = '../../data'
THREADS = 8

## Loading data

In [3]:
test_df = pd.read_csv(f'{DATA_ROOT}/ratings_test_implicit.csv.gz')
train_df = pd.read_csv(f'{DATA_ROOT}/ratings_train_implicit.csv.gz')
games_df = pd.read_json(f'{DATA_ROOT}/bgg_GameItem.jl', lines=True)[[
    'name', 'bgg_id', 'mechanic', 'category', 'complexity',
    'max_players_best', 'min_players_best', 'max_players_rec', 'min_players_rec'
]]

features_names = pd.read_csv(f'{DATA_ROOT}/game_features_names.csv.gz').values.flatten()
game_features = pd.read_csv(f'{DATA_ROOT}/game_features.csv.gz')

In [4]:
users_profiles = pd.read_pickle(f'{DATA_ROOT}/users_profiles.pkl')
games_profiles = pd.read_pickle(f'{DATA_ROOT}/games_profiles.pkl')

In [5]:
mechanics_names = features_names[:20]
categories_names = features_names[20:40]

## Adding new users

In [6]:
USERS_TO_ADD = {
    # Family: Catan, Monopoly, UNO, Ticket to Ride
    "USER_1": [13, 1406, 2223, 9209],
    # Negative interactions: Game of Thrones, War of the Ring, Twilight Imperium (3rd edition),
    # Battlestar Galactica, Scythe
    "USER_2": [103343, 2228, 12493, 37111, 169786],
    # Worker placement: Viticulture, Agricola, Feast for Odin, West Kingdom
    "USER_3": [183394, 31260, 177736, 236457, 266810, 296151],
    # Our games: Spirit Island, Citadel, Splendor, Avalon, Codenames, Crew, Love Letter, 7 wonders, viticulture, Descent
    "USER_4": [162886, 478, 148228, 1862, 178900, 324856, 129622, 68448, 183394, 150586],
    # Ameritrash: Runewars, Arkham Horror, Starcraft, Last Night on Earth, Talisman, Divine Right
    "USER_5": [59294, 15987, 22827, 29368, 27627, 23],
    # Eurogames: Catan, Puerto Rico, Carcassone, Ra, El Grande, Five Tribes
    "USER_6": [13, 3076, 822, 12, 93, 157354],
}

In [7]:
def append_users_to_train_df(train_df, users):
    for user, games in users.items():
        train_df = train_df.append(pd.DataFrame(data={'bgg_user_name': [user]*len(games), 'bgg_id': games, 'value':[1]*len(games)}))
    return train_df

In [8]:
train_df = append_users_to_train_df(train_df, USERS_TO_ADD)
profiles_to_add = create_users_profiles_embeddings(
    train_df, games_df, categories_names, mechanics_names, 10, show_progress=True, users_subset=[f'USER_{i}' for i in range(1,7)]
)
users_profiles = users_profiles.append(profiles_to_add)

100%|██████████| 6/6 [00:00<00:00, 223.11it/s]


## Preparing interactions, item features & profiles

In [9]:
full_df = pd.concat([train_df, test_df], ignore_index=True)
dataset = Dataset()
print("Fitting dataset")
dataset.fit((x for x in full_df['bgg_user_name']), (x for x in full_df['bgg_id']),
            item_features=(x for x in features_names))

Fitting dataset


In [10]:
game_features['features'] = game_features.features.apply(literal_eval)
games_list = full_df['bgg_id'].unique()
game_features = game_features.drop((game_features['bgg_id'])[~game_features['bgg_id'].isin(games_list)].index)
item_features = dataset.build_item_features((val['bgg_id'], val['features'] + [val['bgg_id']])
                                        for idx, val in game_features.iterrows())

In [11]:
try:
    train_interactions = scipy.sparse.load_npz('../notebooks_data/lightfm_train_interactions.npz')
    test_interactions = scipy.sparse.load_npz('../notebooks_data/lightfm_test_interactions.npz')
except:
    train_interactions, test_interactions = prepare_interactions(train_df, test_df, dataset)
    scipy.sparse.save_npz('../notebooks_data/lightfm_train_interactions.npz', train_interactions)
    scipy.sparse.save_npz('../notebooks_data/lightfm_test_interactions.npz', test_interactions)

Preparing training interactions
Preparing testing interactions


In [12]:
users_profiles.index = users_profiles.index.map(dataset.mapping()[0])
games_profiles.index = games_profiles.index.map(dataset.mapping()[2])
users_profiles = users_profiles.sort_index()
games_profiles = games_profiles.sort_index()

In [13]:
users_profiles.values[:, :40] *= 3

In [14]:
cython_users_profiles = np.ascontiguousarray(users_profiles.values, dtype=np.float32)
cython_games_profiles = np.ascontiguousarray(games_profiles.values, dtype=np.float32)

## Model training

In [24]:
PARAMS = {'no_components': 70,
          'learning_schedule': 'adadelta',
          'loss': 'warp',
          'item_alpha': 3e-06,
          'max_sampled': 10,
          'rho': 0.95,
          'epsilon': 1.0e-06,
          'random_state': 42}

model = LightFM(**PARAMS)
model.fit_partial(
    train_interactions, verbose=True, item_features=item_features, epochs=104,
    num_threads=THREADS, user_profiles=cython_users_profiles, item_profiles=cython_games_profiles
)

Epoch: 100%|██████████| 104/104 [1:34:13<00:00, 54.36s/it]


## Inspecting predictions

In [33]:
tests = LightFMTests(model=model,
                     train_df=train_df,
                     dataset=dataset,
                     train_interactions=train_interactions,
                     test_interactions=test_interactions,
                     item_features=item_features)

In [34]:
def analyze(user_name):
    def game_id_to_name(game_id, games_df):
        game_info = games_df.loc[games_df['bgg_id'] == game_id]
        return game_info
    top_5 = tests.get_top_n_for_user_by_name(user_name=user_name, training_games_ids=USERS_TO_ADD[user_name], n=5)
    top_20 = tests.get_top_n_for_user_by_name(user_name=user_name, training_games_ids=USERS_TO_ADD[user_name], n=20)
    top_5_df = pd.DataFrame({"bgg_user_name": user_name, "bgg_id": top_5}).explode('bgg_id').reset_index(drop=True)
    recommendations_profile = create_recommendations_profiles_embeddings(top_5_df, 5, games_df, categories_names, mechanics_names, 10)
    print("Metric value:", calculate_metric_scores(recommendations_profile, users_profiles.iloc[dataset.mapping()[0][user_name]:dataset.mapping()[0][user_name]+1]))
    top_20_names = [game_id_to_name(game_id, games_df) for game_id in top_20]
    return pd.concat(top_20_names)

In [35]:
# Family: Catan, Monopoly, UNO, Ticket to Ride
analyze("USER_1")

Metric value: [0.45       0.72857143 0.43333333 0.77702619 2.38893096]


,name,bgg_id,mechanic,category,complexity,max_players_best,min_players_best,max_players_rec,min_players_rec
720,Carcassonne,822,"[Area Majority / Influence:2080, Map Addition:...","[City Building:1029, Medieval:1035, Territory ...",1.9079,2.0,2.0,5.0,2.0
26601,Pandemic,30549,"[Action Points:2001, Cooperative Game:2023, Ha...",[Medical:2145],2.4080,4.0,4.0,4.0,1.0
163,Risk,181,"[Area Movement:2046, Dice Rolling:2072, Player...","[Territory Building:1086, Wargame:1019]",2.0801,4.0,4.0,6.0,3.0
41983,7 Wonders,68448,"[Drafting:2984, Hand Management:2040, Set Coll...","[Ancient:1050, Card Game:1002, City Building:1...",2.3268,5.0,4.0,7.0,3.0
284,Scrabble,320,"[End Game Bonuses:2875, Hand Management:2040, ...",[Word Game:1025],2.0878,2.0,2.0,4.0,2.0
1092,Clue,1294,"[Deduction:3002, Dice Rolling:2072, Grid Movem...","[Bluffing:1023, Deduction:1039, Movies / TV / ...",1.6565,5.0,4.0,6.0,3.0
13159,Ticket to Ride: Europe,14996,"[Card Drafting:2041, Connections:2883, End Gam...",[Trains:1034],1.9332,4.0,4.0,5.0,2.0
54049,Love Letter,129622,"[Deduction:3002, Hand Management:2040, Player ...","[Card Game:1002, Deduction:1039, Renaissance:1...",1.1920,4.0,4.0,4.0,3.0
1634,Munchkin,1927,"[Hand Management:2040, Take That:2686, Variabl...","[Card Game:1002, Fantasy:1010, Fighting:1046, ...",1.8026,4.0,4.0,6.0,3.0
42497,King of Tokyo,70323,"[Card Drafting:2041, Dice Rolling:2072, Die Ic...","[Dice:1017, Fighting:1046, Movies / TV / Radio...",1.4919,5.0,4.0,6.0,3.0


Metric value: [0.45       0.72857143 0.43333333 0.77702619 2.38893096]


,name,bgg_id,mechanic,category,complexity,max_players_best,min_players_best,max_players_rec,min_players_rec
720,Carcassonne,822,"[Area Majority / Influence:2080, Map Addition:...","[City Building:1029, Medieval:1035, Territory ...",1.9079,2.0,2.0,5.0,2.0
26601,Pandemic,30549,"[Action Points:2001, Cooperative Game:2023, Ha...",[Medical:2145],2.4080,4.0,4.0,4.0,1.0
163,Risk,181,"[Area Movement:2046, Dice Rolling:2072, Player...","[Territory Building:1086, Wargame:1019]",2.0801,4.0,4.0,6.0,3.0
41983,7 Wonders,68448,"[Drafting:2984, Hand Management:2040, Set Coll...","[Ancient:1050, Card Game:1002, City Building:1...",2.3268,5.0,4.0,7.0,3.0
284,Scrabble,320,"[End Game Bonuses:2875, Hand Management:2040, ...",[Word Game:1025],2.0878,2.0,2.0,4.0,2.0
1092,Clue,1294,"[Deduction:3002, Dice Rolling:2072, Grid Movem...","[Bluffing:1023, Deduction:1039, Movies / TV / ...",1.6565,5.0,4.0,6.0,3.0
13159,Ticket to Ride: Europe,14996,"[Card Drafting:2041, Connections:2883, End Gam...",[Trains:1034],1.9332,4.0,4.0,5.0,2.0
54049,Love Letter,129622,"[Deduction:3002, Hand Management:2040, Player ...","[Card Game:1002, Deduction:1039, Renaissance:1...",1.1920,4.0,4.0,4.0,3.0
1634,Munchkin,1927,"[Hand Management:2040, Take That:2686, Variabl...","[Card Game:1002, Fantasy:1010, Fighting:1046, ...",1.8026,4.0,4.0,6.0,3.0
42497,King of Tokyo,70323,"[Card Drafting:2041, Dice Rolling:2072, Die Ic...","[Dice:1017, Fighting:1046, Movies / TV / Radio...",1.4919,5.0,4.0,6.0,3.0


In [36]:
# Negative interactions: Game of Thrones, War of the Ring, Twilight Imperium (3rd edition),
# Battlestar Galactica, Scythe
analyze("USER_2")

Metric value: [0.4        0.83571429 0.16       0.39161042 1.7873247 ]


,name,bgg_id,mechanic,category,complexity,max_players_best,min_players_best,max_players_rec,min_players_rec
10825,Twilight Struggle,12333,"[Action/Event:2840, Advantage Token:2847, Area...","[Modern Warfare:1069, Political:1001, Wargame:...",3.5890,2.0,2.0,2.0,2.0
720,Carcassonne,822,"[Area Majority / Influence:2080, Map Addition:...","[City Building:1029, Medieval:1035, Territory ...",1.9079,2.0,2.0,5.0,2.0
12,Catan,13,"[Dice Rolling:2072, Hexagon Grid:2026, Income:...","[Economic:1021, Negotiation:1026]",2.3152,4.0,4.0,4.0,3.0
64197,Terraforming Mars,167791,"[Drafting:2984, End Game Bonuses:2875, Hand Ma...","[Economic:1021, Environmental:1084, Industry /...",3.2428,3.0,3.0,4.0,1.0
26601,Pandemic,30549,"[Action Points:2001, Cooperative Game:2023, Ha...",[Medical:2145],2.4080,4.0,4.0,4.0,1.0
41983,7 Wonders,68448,"[Drafting:2984, Hand Management:2040, Set Coll...","[Ancient:1050, Card Game:1002, City Building:1...",2.3268,5.0,4.0,7.0,3.0
78949,Root,237182,"[Action Queue:2689, Action Retrieval:2839, Are...","[Animals:1089, Fantasy:1010, Wargame:1019]",3.6998,4.0,4.0,4.0,3.0
423,Diplomacy,483,"[Area Majority / Influence:2080, Area Movement...","[Bluffing:1023, Negotiation:1026, Political:10...",3.3566,7.0,7.0,7.0,6.0
35328,Small World,40692,"[Area Majority / Influence:2080, Area Movement...","[Fantasy:1010, Fighting:1046, Territory Buildi...",2.3543,4.0,4.0,5.0,2.0
27198,Agricola,31260,"[Advantage Token:2847, Automatic Resource Grow...","[Animals:1089, Economic:1021, Farming:1013]",3.6396,4.0,3.0,5.0,1.0


Metric value: [0.4        0.83571429 0.16       0.39161042 1.7873247 ]


,name,bgg_id,mechanic,category,complexity,max_players_best,min_players_best,max_players_rec,min_players_rec
10825,Twilight Struggle,12333,"[Action/Event:2840, Advantage Token:2847, Area...","[Modern Warfare:1069, Political:1001, Wargame:...",3.5890,2.0,2.0,2.0,2.0
720,Carcassonne,822,"[Area Majority / Influence:2080, Map Addition:...","[City Building:1029, Medieval:1035, Territory ...",1.9079,2.0,2.0,5.0,2.0
12,Catan,13,"[Dice Rolling:2072, Hexagon Grid:2026, Income:...","[Economic:1021, Negotiation:1026]",2.3152,4.0,4.0,4.0,3.0
64197,Terraforming Mars,167791,"[Drafting:2984, End Game Bonuses:2875, Hand Ma...","[Economic:1021, Environmental:1084, Industry /...",3.2428,3.0,3.0,4.0,1.0
26601,Pandemic,30549,"[Action Points:2001, Cooperative Game:2023, Ha...",[Medical:2145],2.4080,4.0,4.0,4.0,1.0
41983,7 Wonders,68448,"[Drafting:2984, Hand Management:2040, Set Coll...","[Ancient:1050, Card Game:1002, City Building:1...",2.3268,5.0,4.0,7.0,3.0
78949,Root,237182,"[Action Queue:2689, Action Retrieval:2839, Are...","[Animals:1089, Fantasy:1010, Wargame:1019]",3.6998,4.0,4.0,4.0,3.0
423,Diplomacy,483,"[Area Majority / Influence:2080, Area Movement...","[Bluffing:1023, Negotiation:1026, Political:10...",3.3566,7.0,7.0,7.0,6.0
35328,Small World,40692,"[Area Majority / Influence:2080, Area Movement...","[Fantasy:1010, Fighting:1046, Territory Buildi...",2.3543,4.0,4.0,5.0,2.0
27198,Agricola,31260,"[Advantage Token:2847, Automatic Resource Grow...","[Animals:1089, Economic:1021, Farming:1013]",3.6396,4.0,3.0,5.0,1.0


In [37]:
# Worker placement: Viticulture, Agricola, Feast for Odin, West Kingdom
analyze("USER_3")

Metric value: [1.5       1.35      0.4       0.4080496 3.6580496]


,name,bgg_id,mechanic,category,complexity,max_players_best,min_players_best,max_players_rec,min_players_rec
64197,Terraforming Mars,167791,"[Drafting:2984, End Game Bonuses:2875, Hand Ma...","[Economic:1021, Environmental:1084, Industry /...",3.2428,3.0,3.0,4.0,1.0
85041,Wingspan,266192,"[Card Drafting:2041, Dice Rolling:2072, Drafti...","[Animals:1089, Card Game:1002, Educational:1094]",2.4307,3.0,3.0,4.0,1.0
64757,Raiders of the North Sea,170042,"[Contracts:2912, Dice Rolling:2072, Hand Manag...","[Medieval:1035, Nautical:1008]",2.5283,3.0,3.0,4.0,2.0
41983,7 Wonders,68448,"[Drafting:2984, Hand Management:2040, Set Coll...","[Ancient:1050, Card Game:1002, City Building:1...",2.3268,5.0,4.0,7.0,3.0
65698,7 Wonders Duel,173346,"[Card Drafting:2041, Layering:3001, Set Collec...","[Ancient:1050, Card Game:1002, City Building:1...",2.2258,2.0,2.0,2.0,2.0
64700,Scythe,169786,"[Action Drafting:2838, Card Play Conflict Reso...","[Economic:1021, Fighting:1046, Miniatures:1047...",3.4225,4.0,4.0,5.0,1.0
720,Carcassonne,822,"[Area Majority / Influence:2080, Map Addition:...","[City Building:1029, Medieval:1035, Territory ...",1.9079,2.0,2.0,5.0,2.0
26601,Pandemic,30549,"[Action Points:2001, Cooperative Game:2023, Ha...",[Medical:2145],2.4080,4.0,4.0,4.0,1.0
70805,Great Western Trail,193738,"[Deck, Bag, and Pool Building:2664, Hand Manag...","[American West:1055, Animals:1089, Economic:1021]",3.7090,3.0,3.0,4.0,2.0
47735,Caverna: The Cave Farmers,102794,"[Automatic Resource Growth:2903, Increase Valu...","[Animals:1089, Economic:1021, Fantasy:1010, Fa...",3.7898,4.0,4.0,5.0,1.0


Metric value: [1.5       1.35      0.4       0.4080496 3.6580496]


,name,bgg_id,mechanic,category,complexity,max_players_best,min_players_best,max_players_rec,min_players_rec
64197,Terraforming Mars,167791,"[Drafting:2984, End Game Bonuses:2875, Hand Ma...","[Economic:1021, Environmental:1084, Industry /...",3.2428,3.0,3.0,4.0,1.0
85041,Wingspan,266192,"[Card Drafting:2041, Dice Rolling:2072, Drafti...","[Animals:1089, Card Game:1002, Educational:1094]",2.4307,3.0,3.0,4.0,1.0
64757,Raiders of the North Sea,170042,"[Contracts:2912, Dice Rolling:2072, Hand Manag...","[Medieval:1035, Nautical:1008]",2.5283,3.0,3.0,4.0,2.0
41983,7 Wonders,68448,"[Drafting:2984, Hand Management:2040, Set Coll...","[Ancient:1050, Card Game:1002, City Building:1...",2.3268,5.0,4.0,7.0,3.0
65698,7 Wonders Duel,173346,"[Card Drafting:2041, Layering:3001, Set Collec...","[Ancient:1050, Card Game:1002, City Building:1...",2.2258,2.0,2.0,2.0,2.0
64700,Scythe,169786,"[Action Drafting:2838, Card Play Conflict Reso...","[Economic:1021, Fighting:1046, Miniatures:1047...",3.4225,4.0,4.0,5.0,1.0
720,Carcassonne,822,"[Area Majority / Influence:2080, Map Addition:...","[City Building:1029, Medieval:1035, Territory ...",1.9079,2.0,2.0,5.0,2.0
26601,Pandemic,30549,"[Action Points:2001, Cooperative Game:2023, Ha...",[Medical:2145],2.4080,4.0,4.0,4.0,1.0
70805,Great Western Trail,193738,"[Deck, Bag, and Pool Building:2664, Hand Manag...","[American West:1055, Animals:1089, Economic:1021]",3.7090,3.0,3.0,4.0,2.0
47735,Caverna: The Cave Farmers,102794,"[Automatic Resource Growth:2903, Increase Valu...","[Animals:1089, Economic:1021, Fantasy:1010, Fa...",3.7898,4.0,4.0,5.0,1.0


In [38]:
# Our games: Spirit Island, Citadel, Splendor, Avalon, Codenames, Crew, Love Letter, 7 wonders, viticulture, Descent
analyze("USER_4")

Metric value: [0.12857143 1.15384615 0.25882353 0.63361439 2.1748555 ]


,name,bgg_id,mechanic,category,complexity,max_players_best,min_players_best,max_players_rec,min_players_rec
62819,Pandemic Legacy: Season 1,161936,"[Action Points:2001, Cooperative Game:2023, Ha...","[Environmental:1084, Medical:2145]",2.8336,4.0,4.0,4.0,2.0
720,Carcassonne,822,"[Area Majority / Influence:2080, Map Addition:...","[City Building:1029, Medieval:1035, Territory ...",1.9079,2.0,2.0,5.0,2.0
26601,Pandemic,30549,"[Action Points:2001, Cooperative Game:2023, Ha...",[Medical:2145],2.4080,4.0,4.0,4.0,1.0
12,Catan,13,"[Dice Rolling:2072, Hexagon Grid:2026, Income:...","[Economic:1021, Negotiation:1026]",2.3152,4.0,4.0,4.0,3.0
75726,Pandemic Legacy: Season 2,221107,"[Cooperative Game:2023, Hand Management:2040, ...","[Environmental:1084, Exploration:1020, Medical...",3.2292,4.0,4.0,4.0,2.0
65698,7 Wonders Duel,173346,"[Card Drafting:2041, Layering:3001, Set Collec...","[Ancient:1050, Card Game:1002, City Building:1...",2.2258,2.0,2.0,2.0,2.0
46988,Hanabi,98778,"[Communication Limits:2893, Cooperative Game:2...","[Card Game:1002, Deduction:1039, Memory:1045]",1.6997,4.0,4.0,5.0,2.0
128,Pit,140,"[Set Collection:2004, Trading:2008]","[Card Game:1002, Economic:1021, Negotiation:10...",1.1590,8.0,6.0,9.0,4.0
712,Rummikub,811,"[Melding and Splaying:2981, Set Collection:200...","[Abstract Strategy:1009, Card Game:1002, Numbe...",1.7413,4.0,4.0,4.0,2.0
63217,Patchwork,163412,"[Card Drafting:2041, Grid Coverage:2978, Incom...","[Abstract Strategy:1009, Puzzle:1028]",1.6252,2.0,2.0,2.0,2.0


Metric value: [0.12857143 1.15384615 0.25882353 0.63361439 2.1748555 ]


,name,bgg_id,mechanic,category,complexity,max_players_best,min_players_best,max_players_rec,min_players_rec
62819,Pandemic Legacy: Season 1,161936,"[Action Points:2001, Cooperative Game:2023, Ha...","[Environmental:1084, Medical:2145]",2.8336,4.0,4.0,4.0,2.0
720,Carcassonne,822,"[Area Majority / Influence:2080, Map Addition:...","[City Building:1029, Medieval:1035, Territory ...",1.9079,2.0,2.0,5.0,2.0
26601,Pandemic,30549,"[Action Points:2001, Cooperative Game:2023, Ha...",[Medical:2145],2.4080,4.0,4.0,4.0,1.0
12,Catan,13,"[Dice Rolling:2072, Hexagon Grid:2026, Income:...","[Economic:1021, Negotiation:1026]",2.3152,4.0,4.0,4.0,3.0
75726,Pandemic Legacy: Season 2,221107,"[Cooperative Game:2023, Hand Management:2040, ...","[Environmental:1084, Exploration:1020, Medical...",3.2292,4.0,4.0,4.0,2.0
65698,7 Wonders Duel,173346,"[Card Drafting:2041, Layering:3001, Set Collec...","[Ancient:1050, Card Game:1002, City Building:1...",2.2258,2.0,2.0,2.0,2.0
46988,Hanabi,98778,"[Communication Limits:2893, Cooperative Game:2...","[Card Game:1002, Deduction:1039, Memory:1045]",1.6997,4.0,4.0,5.0,2.0
128,Pit,140,"[Set Collection:2004, Trading:2008]","[Card Game:1002, Economic:1021, Negotiation:10...",1.1590,8.0,6.0,9.0,4.0
712,Rummikub,811,"[Melding and Splaying:2981, Set Collection:200...","[Abstract Strategy:1009, Card Game:1002, Numbe...",1.7413,4.0,4.0,4.0,2.0
63217,Patchwork,163412,"[Card Drafting:2041, Grid Coverage:2978, Incom...","[Abstract Strategy:1009, Puzzle:1028]",1.6252,2.0,2.0,2.0,2.0


In [39]:
# Ameritrash: Runewars, Arkham Horror, Starcraft, Last Night on Earth, Talisman, Divine Right
analyze("USER_5")

Metric value: [1.         1.48       0.325      0.79938471 3.60438471]


,name,bgg_id,mechanic,category,complexity,max_players_best,min_players_best,max_players_rec,min_players_rec
10959,Twilight Imperium: Third Edition,12493,"[Action Points:2001, Area-Impulse:2021, Dice R...","[Civilization:1015, Negotiation:1026, Politica...",4.2535,6.0,6.0,6.0,4.0
10825,Twilight Struggle,12333,"[Action/Event:2840, Advantage Token:2847, Area...","[Modern Warfare:1069, Political:1001, Wargame:...",3.5890,2.0,2.0,2.0,2.0
47844,A Game of Thrones: The Board Game (Second Edit...,103343,"[Area Majority / Influence:2080, Area Movement...","[Bluffing:1023, Fantasy:1010, Negotiation:1026...",3.7218,6.0,6.0,6.0,4.0
9229,Betrayal at House on the Hill,10547,"[Dice Rolling:2072, Map Addition:2959, Modular...","[Adventure:1022, Exploration:1020, Horror:1024...",2.3870,6.0,5.0,6.0,3.0
42845,Eclipse,72125,"[Area Majority / Influence:2080, Dice Rolling:...","[Civilization:1015, Fighting:1046, Science Fic...",3.7002,6.0,4.0,6.0,2.0
9305,Memoir '44,10630,"[Area Majority / Influence:2080, Campaign / Ba...","[Miniatures:1047, Wargame:1019, World War II:1...",2.2714,2.0,2.0,2.0,2.0
19852,Tide of Iron,22825,"[Action Points:2001, Dice Rolling:2072, Hand M...","[Miniatures:1047, Wargame:1019, World War II:1...",3.4577,2.0,2.0,4.0,2.0
12,Catan,13,"[Dice Rolling:2072, Hexagon Grid:2026, Income:...","[Economic:1021, Negotiation:1026]",2.3152,4.0,4.0,4.0,3.0
15101,Descent: Journeys in the Dark,17226,"[Dice Rolling:2072, Grid Movement:2676, Modula...","[Adventure:1022, Exploration:1020, Fantasy:101...",3.3409,5.0,5.0,5.0,2.0
94,Titan,103,"[Dice Rolling:2072, Minimap Resolution:2863, P...","[Fantasy:1010, Wargame:1019]",3.4573,4.0,4.0,6.0,2.0


Metric value: [1.         1.48       0.325      0.79938471 3.60438471]


,name,bgg_id,mechanic,category,complexity,max_players_best,min_players_best,max_players_rec,min_players_rec
10959,Twilight Imperium: Third Edition,12493,"[Action Points:2001, Area-Impulse:2021, Dice R...","[Civilization:1015, Negotiation:1026, Politica...",4.2535,6.0,6.0,6.0,4.0
10825,Twilight Struggle,12333,"[Action/Event:2840, Advantage Token:2847, Area...","[Modern Warfare:1069, Political:1001, Wargame:...",3.5890,2.0,2.0,2.0,2.0
47844,A Game of Thrones: The Board Game (Second Edit...,103343,"[Area Majority / Influence:2080, Area Movement...","[Bluffing:1023, Fantasy:1010, Negotiation:1026...",3.7218,6.0,6.0,6.0,4.0
9229,Betrayal at House on the Hill,10547,"[Dice Rolling:2072, Map Addition:2959, Modular...","[Adventure:1022, Exploration:1020, Horror:1024...",2.3870,6.0,5.0,6.0,3.0
42845,Eclipse,72125,"[Area Majority / Influence:2080, Dice Rolling:...","[Civilization:1015, Fighting:1046, Science Fic...",3.7002,6.0,4.0,6.0,2.0
9305,Memoir '44,10630,"[Area Majority / Influence:2080, Campaign / Ba...","[Miniatures:1047, Wargame:1019, World War II:1...",2.2714,2.0,2.0,2.0,2.0
19852,Tide of Iron,22825,"[Action Points:2001, Dice Rolling:2072, Hand M...","[Miniatures:1047, Wargame:1019, World War II:1...",3.4577,2.0,2.0,4.0,2.0
12,Catan,13,"[Dice Rolling:2072, Hexagon Grid:2026, Income:...","[Economic:1021, Negotiation:1026]",2.3152,4.0,4.0,4.0,3.0
15101,Descent: Journeys in the Dark,17226,"[Dice Rolling:2072, Grid Movement:2676, Modula...","[Adventure:1022, Exploration:1020, Fantasy:101...",3.3409,5.0,5.0,5.0,2.0
94,Titan,103,"[Dice Rolling:2072, Minimap Resolution:2863, P...","[Fantasy:1010, Wargame:1019]",3.4573,4.0,4.0,6.0,2.0


In [40]:
# Eurogames: Catan, Puerto Rico, Carcassone, Ra, El Grande, Five Tribes
analyze("USER_6")

Metric value: [0.6        0.72631579 0.48571429 0.73061489 2.54264497]


,name,bgg_id,mechanic,category,complexity,max_players_best,min_players_best,max_players_rec,min_players_rec
41983,7 Wonders,68448,"[Drafting:2984, Hand Management:2040, Set Coll...","[Ancient:1050, Card Game:1002, City Building:1...",2.3268,5.0,4.0,7.0,3.0
2279,Power Grid,2651,"[Auction/Bidding:2012, Auction: Turn Order Unt...","[Economic:1021, Industry / Manufacturing:1088]",3.2682,5.0,4.0,6.0,3.0
39,Tigris & Euphrates,42,"[Area Majority / Influence:2080, Connections:2...","[Ancient:1050, Civilization:1015, Territory Bu...",3.5079,4.0,4.0,4.0,2.0
26601,Pandemic,30549,"[Action Points:2001, Cooperative Game:2023, Ha...",[Medical:2145],2.4080,4.0,4.0,4.0,1.0
8025,Ticket to Ride,9209,"[Card Drafting:2041, Contracts:2912, End Game ...",[Trains:1034],1.8456,4.0,4.0,5.0,2.0
13159,Ticket to Ride: Europe,14996,"[Card Drafting:2041, Connections:2883, End Gam...",[Trains:1034],1.9332,4.0,4.0,5.0,2.0
27198,Agricola,31260,"[Advantage Token:2847, Automatic Resource Grow...","[Animals:1089, Economic:1021, Farming:1013]",3.6396,4.0,3.0,5.0,1.0
59174,Splendor,148228,"[Card Drafting:2041, Contracts:2912, Race:2876...","[Card Game:1002, Economic:1021, Renaissance:1070]",1.7948,3.0,3.0,4.0,2.0
35328,Small World,40692,"[Area Majority / Influence:2080, Area Movement...","[Fantasy:1010, Fighting:1046, Territory Buildi...",2.3543,4.0,4.0,5.0,2.0
10,Bohnanza,11,"[Hand Management:2040, Melding and Splaying:29...","[Card Game:1002, Farming:1013, Negotiation:1026]",1.6756,5.0,4.0,7.0,3.0


Metric value: [0.6        0.72631579 0.48571429 0.73061489 2.54264497]


,name,bgg_id,mechanic,category,complexity,max_players_best,min_players_best,max_players_rec,min_players_rec
41983,7 Wonders,68448,"[Drafting:2984, Hand Management:2040, Set Coll...","[Ancient:1050, Card Game:1002, City Building:1...",2.3268,5.0,4.0,7.0,3.0
2279,Power Grid,2651,"[Auction/Bidding:2012, Auction: Turn Order Unt...","[Economic:1021, Industry / Manufacturing:1088]",3.2682,5.0,4.0,6.0,3.0
39,Tigris & Euphrates,42,"[Area Majority / Influence:2080, Connections:2...","[Ancient:1050, Civilization:1015, Territory Bu...",3.5079,4.0,4.0,4.0,2.0
26601,Pandemic,30549,"[Action Points:2001, Cooperative Game:2023, Ha...",[Medical:2145],2.4080,4.0,4.0,4.0,1.0
8025,Ticket to Ride,9209,"[Card Drafting:2041, Contracts:2912, End Game ...",[Trains:1034],1.8456,4.0,4.0,5.0,2.0
13159,Ticket to Ride: Europe,14996,"[Card Drafting:2041, Connections:2883, End Gam...",[Trains:1034],1.9332,4.0,4.0,5.0,2.0
27198,Agricola,31260,"[Advantage Token:2847, Automatic Resource Grow...","[Animals:1089, Economic:1021, Farming:1013]",3.6396,4.0,3.0,5.0,1.0
59174,Splendor,148228,"[Card Drafting:2041, Contracts:2912, Race:2876...","[Card Game:1002, Economic:1021, Renaissance:1070]",1.7948,3.0,3.0,4.0,2.0
35328,Small World,40692,"[Area Majority / Influence:2080, Area Movement...","[Fantasy:1010, Fighting:1046, Territory Buildi...",2.3543,4.0,4.0,5.0,2.0
10,Bohnanza,11,"[Hand Management:2040, Melding and Splaying:29...","[Card Game:1002, Farming:1013, Negotiation:1026]",1.6756,5.0,4.0,7.0,3.0
